<a href="https://colab.research.google.com/github/dvircohen0/Machine-Learning-Algorithms-From-Scratch/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://iamtrask.github.io/img/recurrence_gif.gif)

In [30]:
import numpy as np
from tqdm import tqdm

 
class RNN:
    def __init__(self, input_dim=2, output_dim=1, hidden_dim=16):
        self.input_weights = 2*np.random.random((input_dim,hidden_dim)) - 1
        self.output_weights = 2*np.random.random((hidden_dim,output_dim)) - 1
        self.hidden_weights = 2*np.random.random((hidden_dim,hidden_dim)) - 1
        self.binary_dim = 8
        self.hidden_dim=hidden_dim
        self.int2binary=self.int_to_binary()

        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def sigmoid_output_to_derivative(self,x):
        return x*(1-x)
        
    def int_to_binary(self):
        int2binary={}
        self.largest_number = 2**self.binary_dim
        binary = np.unpackbits( np.array([range(self.largest_number)],dtype=np.uint8).T,axis=1)
        for i in range(self.largest_number):
            int2binary[i] = binary[i]
        return int2binary
    
    def generate_random(self):
        self.a_int = np.random.randint(self.largest_number/2)
        self.a = self.int2binary[self.a_int]
        self.b_int = np.random.randint(self.largest_number/2)
        self.b = self.int2binary[self.b_int]
        self.c = self.int2binary[self.a_int + self.b_int]

    
    def forward(self):
        self.total_error=0
        self.layer_2_deltas=[]
        self.layer_1_values = [np.expand_dims(np.zeros(self.hidden_dim),axis=0)]
        for pos in range(self.binary_dim):
            input_numbers = np.array([[self.a[self.binary_dim - pos - 1],self.b[self.binary_dim - pos -1]]])
            result = np.array([self.c[self.binary_dim - pos -1]])
            layer_1 = self.sigmoid(np.dot(input_numbers,self.input_weights)+np.dot(self.layer_1_values[-1],self.hidden_weights))
            layer_2 = self.sigmoid(np.dot(layer_1,self.output_weights))
            self.layer_1_values.append(layer_1 )
            error = result - layer_2
            self.layer_2_deltas.append((error)*self.sigmoid_output_to_derivative(layer_2))
            self.total_error += abs(error[0][0])
            self.d[self.binary_dim - pos - 1] = np.round(layer_2[0][0])

            
    def backward(self):
        self.input_weights_update=0
        self.output_weights_update=0
        self.hidden_weights_update=0
        future_layer_1_delta = np.zeros(self.hidden_dim)
        for pos in range(self.binary_dim):
            X = np.array([[self.a[pos],self.b[pos]]])
            layer_1 = self.layer_1_values[self.binary_dim-pos]
            prev_layer_1 = self.layer_1_values[self.binary_dim-pos-1]
            layer_2_delta = self.layer_2_deltas[self.binary_dim-pos-1]
            layer_1_delta = (np.dot(future_layer_1_delta,self.hidden_weights) + \
                             np.dot(layer_2_delta,self.output_weights.T)) *     \
                              self.sigmoid_output_to_derivative(layer_1)
                              
            self.output_weights_update += np.dot(layer_1.T,layer_2_delta)
            self.hidden_weights_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
            self.input_weights_update += X.T.dot(layer_1_delta)
            future_layer_1_delta=layer_1_delta
            
            
    def weights_update(self):
        self.input_weights += self.input_weights_update * self.lr
        self.output_weights += self.output_weights_update * self.lr
        self.hidden_weights += self.hidden_weights_update * self.lr


    def train(self, lr=0.1,epoches=10):
        self.lr=lr
        for j in range(epoches):
            self.generate_random()
            self.d = np.zeros_like(self.c)
            self.forward()
            self.backward()
            self.weights_update()
            if (j % 1000 == 0):
              print("Epoch: ",str(j))
              print("Pred:",str(self.d))
              print("True:",str(self.c),"\n")
              
            
            
    def predict(self,a,b):
        if a+b> self.largest_number:
            print("please enter numbers with sum lower then ", self.largest_number)
        else:
            self.a_int = a
            self.a = self.int2binary[self.a_int]
            self.b_int = b
            self.b = self.int2binary[self.b_int]
            # self.c = self.int2binary[self.a_int + self.b_int]
            self.forward()
            out=0
            for index,x in enumerate(reversed(self.d)):
                out += x*pow(2,index)
            print(str(a),"+",str(b),"=",str(out))
            if out == self.a_int+self.b_int:
                print("\n",out, "its the right answer!")
            else: print("\n",out, "wrong, the answer is ",self.a_int+self.b_int )

In [35]:
z=RNN()
z.train(0.1,10000)

Epoch:  0
Pred: [0 0 0 0 0 0 0 0]
True: [0 1 1 0 1 1 1 1] 

Epoch:  1000
Pred: [1 0 0 1 0 1 1 1]
True: [0 1 0 0 1 1 0 0] 

Epoch:  2000
Pred: [0 0 0 0 0 0 1 0]
True: [1 0 1 1 0 0 0 0] 

Epoch:  3000
Pred: [0 1 1 1 1 1 0 0]
True: [1 0 0 0 0 0 0 0] 

Epoch:  4000
Pred: [1 0 0 0 0 0 0 0]
True: [1 0 1 0 0 1 0 0] 

Epoch:  5000
Pred: [1 1 0 0 1 0 1 0]
True: [1 0 0 0 1 0 1 0] 

Epoch:  6000
Pred: [0 1 1 0 1 0 1 1]
True: [0 1 1 0 1 0 1 1] 

Epoch:  7000
Pred: [0 1 1 1 0 1 1 1]
True: [0 1 1 1 0 1 1 1] 

Epoch:  8000
Pred: [0 1 0 0 1 1 1 1]
True: [0 1 0 0 1 0 1 1] 

Epoch:  9000
Pred: [1 0 0 0 1 1 0 1]
True: [1 0 0 0 1 1 0 1] 



In [36]:
import random 

a=random.randint(0, 256)
b=random.randint(0, 256-a)

z.predict(a,b)

190 + 7 = 197

 197 its the right answer!
